In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load the data
df = pd.read_excel("./data/FINAL_DATASET.xlsx")

for name in df.columns:
    newname = name
    # remove alla characted within two []
    while "[" in newname:
        start = newname.find("[")
        end = newname.find("]")
        newname = newname[:start] + newname[end+1:]
    # remove end spaces and beginning spaces
    newname = newname.strip()
    # remove alla mathematical oipreators caharacters and spaces
    newname = newname.replace(" ", "_")
    newname = newname.replace(">", "gt")
    newname = newname.replace("<", "lt")
    newname = newname.replace("=", "eq")
    newname = newname.replace("+", "plus")
    newname = newname.replace("-", "minus")
    newname = newname.replace("*", "times")
    newname = newname.replace("/", "div")
    # rename columns in the dataframe
    df.rename(columns={name: newname}, inplace=True)

In [ ]:
labelname  = "d33"
materialfeatures = ['octahedra_volume_min', 'octahedra_volume_max', 'octahedra_volume_avg',
       'octahedra_meanangle_axis_min', 'octahedra_meanangle_axis_max',
       'octahedra_meanangle_axis_avg', 'tilt_BOB_ip_min', 'tilt_BOB_ip_max',
       'tilt_BOB_ip_avg', 'tilt_BOB_oop_min', 'tilt_BOB_oop_max',
       'tilt_BOB_oop_avg', 'spageGroup_no', 'lattice_a', 'lattice_b',
       'lattice_c', 'lattice_alfa', 'lattice_beta', 'lattice_gamma',
       'volume_uc', 'volume_uc_per_atom', 'volume_ratio_ucVSoctahedra',
       'tolerance_factor', 'ratio_outVSinplaneAVG', 'ratio_outVSinplanemin',
       'ratio_outVSinplanemax', 'bond_lengthAA_min', 'bond_lengthAA_max',
       'bond_lengthAA_avg', 'bond_lengthAB_min', 'bond_lengthAB_max',
       'bond_lengthAB_avg', 'bond_lengthAO_min', 'bond_lengthAO_max',
       'bond_lengthAO_avg', 'bond_lengthBO_min', 'bond_lengthBO_max',
       'bond_lengthBO_avg', 'bond_lengthBB_min', 'bond_lengthBB_max',
       'bond_lengthBB_avg', 'is_magnetic', #'is_metallic', 'is_perovskite', 
       #'P_0_z', 
       'P_0_z_muCdivcm2',]
atomicfeatures = ['A_Z', 'A_group', 'A_valence',
       'A_vecdivZ', 'A_n_d', 'A_atomic_volume_pymatgen', 'A_Rdce', 'A_Rdve',
       'A_rs_max', 'A_rd_max', 'A_IE_ionization_energy',
       'A_EA_electron_affinity', 'A_Mulliken', 'A_Pauling',
       'A_MartynovminusBatsanov', 'A_atomic_radius_rahm', 'A_vdw_radius_uff',
       'A_ionic_radius', 'B_Z', 'B_group', 'B_valence', 'B_vecdivZ', 'B_n_d',
       'B_atomic_volume_pymatgen', 'B_Rdce', 'B_Rdve', 'B_rs_max', 'B_rd_max',
       'B_IE_ionization_energy', 'B_EA_electron_affinity', 'B_Mulliken',
       'B_Pauling', 'B_MartynovminusBatsanov', 'B_atomic_radius_rahm',
       'B_vdw_radius_uff', 'B_residual_d', 'B_ionic_radius']

Y = df[labelname]
X = df[materialfeatures + atomicfeatures]
X_mat = df[materialfeatures]
X_atm = df[atomicfeatures]

# classification labbel for the dataset splitted by value
cutval = 5.0
Y_class = []
for y in Y:
    if y < -1.0*cutval:
        Y_class.append(0)
    elif y > -1.0*cutval and y < cutval:
        Y_class.append(1)
    else:
        Y_class.append(2)

Y_class = np.array(Y_class)
# print the number of values for each class
print("Number of values in class 0: ", len(Y_class[Y_class==0]))
print("Number of values in class 1: ", len(Y_class[Y_class==1]))
print("Number of values in class 2: ", len(Y_class[Y_class==2]))


print("min label: ", min(Y))
print("max label: ", max(Y))
print("Number of values   greater than 0: ", len(Y[Y>0]))
print("Number of values     lower than 0: ", len(Y[Y<0]))
print("Number of values       equal to 0: ", len(Y[Y==0]))
print("Number of values   greater than 5: ", len(Y[Y>cutval]))
print("Number of values    lower than -5: ", len(Y[Y<-1.0*cutval]))
print("Number of values between -5 and 5: ", len(Y[(Y>-1.0*cutval) & (Y<cutval)]))

plt.figure(figsize=(10, 10))
plt.hist(Y, bins=100)
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Histogram of the labels")
plt.show()

In [ ]:
CORRCUT = 0.95
corr = X.corr()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
for i in range(upper.shape[0]):
    for j in range(upper.shape[1]):
        if upper.iloc[i,j] > CORRCUT:
            print("%30s %30s %5.2f"%(upper.columns[i], \
                                     upper.columns[j], \
                                        upper.iloc[i,j]))

to_drop = [column for column in upper.columns if any(upper[column] >CORRCUT)]
print("Dropping features: ", to_drop)
X = X.drop(X[to_drop], axis=1)

corr_mat = X_mat.corr()
upper = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > CORRCUT)]
print("Dropping features: ", to_drop)
X_mat = X_mat.drop(X_mat[to_drop], axis=1)

corr_atm = X_atm.corr()
upper = corr_atm.where(np.triu(np.ones(corr_atm.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > CORRCUT)]
print("Dropping features: ", to_drop)
X_atm = X_atm.drop(X_atm[to_drop], axis=1)

In [4]:
# build histograms for the features
#for feature in X.columns:
#    plt.figure(figsize=(10, 10))
#    plt.hist(X[feature], bins=100)
#    plt.xlabel("Value")
#    plt.ylabel("Frequency")
#    plt.title("Histogram of the feature: "+feature)
#    plt.show()

In [ ]:
# build histogram for classes
plt.figure(figsize=(10, 10))
plt.hist(Y_class, bins=3)
plt.xlabel("Class")
plt.ylabel("Frequency")
plt.title("Histogram of the classes")
plt.show()

In [ ]:
# build a classification model using RF 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, Y_train, Y_test = train_test_split(X, Y_class, test_size=0.2, random_state=42) 
accuracies = []
for ntrees in range(10, 200, 10):
    clf = RandomForestClassifier(n_estimators=ntrees, random_state=0)
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    accuracies.append(accuracy)

#plt.figure(figsize=(10, 10))
#plt.plot(range(10, 200, 10), accuracies)
#plt.xlabel("Number of trees")
#plt.ylabel("Accuracy")
#plt.title("Accuracy vs number of trees")
#plt.show()

bestnumoftress = range(10, 200, 10)[accuracies.index(max(accuracies))]
print("Best accuracy: ", max(accuracies), " with ", range(10, 200, 10)[accuracies.index(max(accuracies))], " trees")

In [ ]:
# build a RF classification model using X_atm

X_train, X_test, Y_train, Y_test = train_test_split(X_atm, Y_class, test_size=0.2, random_state=42)
accuracies = []
for ntrees in range(10, 200, 10):
    clf = RandomForestClassifier(n_estimators=ntrees, random_state=0)
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    accuracies.append(accuracy)

#plt.figure(figsize=(10, 10))
#plt.plot(range(10, 200, 10), accuracies)
#plt.xlabel("Number of trees")
#plt.ylabel("Accuracy")
#plt.title("Accuracy vs number of trees")
#plt.show()

print("Best accuracy: ", max(accuracies), " with ", range(10, 200, 10)[accuracies.index(max(accuracies))], " trees")


In [ ]:
# build a RF classification model using X_mat\
X_train, X_test, Y_train, Y_test = train_test_split(X_mat, Y_class, test_size=0.2, random_state=42)
accuracies = []
for ntrees in range(10, 200, 10):
    clf = RandomForestClassifier(n_estimators=ntrees, random_state=0)
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    accuracies.append(accuracy)

print("Best accuracy: ", max(accuracies), " with ", range(10, 200, 10)[accuracies.index(max(accuracies))], " trees")


In [ ]:
# build the RF classification model using X
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_class, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=bestnumoftress, random_state=0)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
testaccuracy = accuracy_score(Y_test, Y_pred)
print("Test accuracy: ", testaccuracy)
Y_pred = clf.predict(X_train)
trainaccuracy = accuracy_score(Y_train, Y_pred)
print("Train accuracy: ", trainaccuracy)
overallaccuracy = accuracy_score(Y_class, clf.predict(X))   
print("Overall accuracy: ", overallaccuracy)

In [ ]:
# print the feature importance not the best approach
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print("%2d) %-*s %f"%(f+1, 30, X.columns[indices[f]], importances[indices[f]]))

In [ ]:
# print the trees of the RF
from sklearn.tree import export_graphviz
import pydot
import graphviz
print("Number of trees: ", len(clf.estimators_))
for i in range(3):
    tree = clf.estimators_[i]
    print("Tree ", i, "Max depth: ", tree.tree_.max_depth)
    dot_data = export_graphviz(tree,
                               feature_names=X.columns,  
                               filled=True,  
                               max_depth=2, 
                               impurity=False, 
                               proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)